## 1. Importación de librerias
_____

In [1]:
import numpy as np
import pickle
import cv2
import tensorflow as tf 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import os 
from os import listdir

## 2. Cargar set de Imágenes
_____

In [ ]:
# Dimensiones de las imágenes
DEFAULT_IMAGE_SIZE = tuple((256, 256))

# Número de imagenes a utilizar
N_IMAGES = 281

# Ruta de carpeta de imágenes
root_dir = 'Fotos'

#train_dir = os.path.join(root_dir, 'train')
#val_dir = os.path.join(root_dir, 'val')

#train_dir = 'Fotos'
#val_dir = 'Fotos'


## 3. Convertir images en arreglos
_____

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, DEFAULT_IMAGE_SIZE)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

## 4. Procesamiento de imágenes
-----
1. Identifación de carpetas 
2. Identificación de imágenes

In [ ]:
image_list, label_list = [], []

try:
    print("[INFO] Procesamiento de imágenes...")
    plant_disease_folder_list = listdir(root_dir)
    
    for plant_disease_folder in plant_disease_folder_list:
        print(f"[INFO] Identificación de carpetas {plant_disease_folder}...")
        
        if(plant_disease_folder != ".DS_Store"):
            plant_disease_image_list = listdir(f"{root_dir}/{plant_disease_folder}/")
            for image in plant_disease_image_list[:N_IMAGES]:
                image_directory = f"{root_dir}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpeg")==True or image_directory.endswith(".JPG")==True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)

    print("[INFO] Procesamiento de imagenes exitoso.")  
except Exception as e:
    print(f"Error : {e}")
    
# Transformación de las imágenes de entrenamiento, cargados en una matriz numpy
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

# Comprobar del número de imágenes cargadas para el entrenamiento
image_len = len(image_list)
print(f"Cantidad de imágenes: {image_len}")

## 5. Etiquetado
-----
1. Identifación de clases
2. Binarización de etiquetas

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)

pickle.dump(label_binarizer,open('plant_disease_label_transform.pkl', 'wb'))

n_classes = len(label_binarizer.classes_)

print("Número de clases de imágenes: ", n_classes)

## 6. Tratamiento de datos
-----
1. El aumento de datos no fue necesario, esto debido a que se obtuvieron mejores resultados sin la necesidad de aumentar las imágenes.

In [ ]:
#augment = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
#                              height_shift_range=0.1, shear_range=0.2, 
#                              zoom_range=0.2, horizontal_flip=True, 
#                              fill_mode="nearest")
    
#augment = ImageDataGenerator()

# img_dir_path = "Fotos/no_sana"
# save_dir_path = "Fotos/train"

# train_generator = augment.flow_from_directory(
#     img_dir_path, 
#     target_size=(128, 128), 
#     color_mode="rgb", 
#     batch_size=20, 
#     save_to_dir="Respaldo", 
#     class_mode="binary", 
#     save_prefix="augmented", 
#     save_format="png")

# for i in range(10):
#     train_generator.next()

In [ ]:
print("[INFO] Dividir el conjunto de datos en datos de entrenamiento y datos de prueba...")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state=45)

In [ ]:
# Establecer parámetros para el modelo
EPOCHS = 20
STEPS = 100
LR = 1e-3
BATCH_SIZE = 16  # Cantidad de muestra por cada época
WIDTH = 256
HEIGHT = 256
DEPTH = 3

## 7. CREAR ARQUITECTURA
-----

In [ ]:
model = Sequential()
inputShape = (HEIGHT, WIDTH, DEPTH)
chanDim = -1

print(K.image_data_format())

#if K.image_data_format() == "channels_first":
#    inputShape = (DEPTH, HEIGHT, WIDTH)
#    chanDim = 1

model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

# Comprobar arquitectura
model.summary()

## 8. Entrenamiento del modelo
-----

In [ ]:
import keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# Activar el optimizador
opt = Adam(learning_rate=LR, decay=LR/EPOCHS)

# Compilar el modelo
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), f1_metric])

# Entrenar el modelo
print("[INFO] Entrenando al modelo...")
history = model.fit(x_train, y_train, batch_size=BATCH_SIZE,
                    validation_data=(x_test, y_test),
                    steps_per_epoch=len(x_train) // BATCH_SIZE,
                    epochs=EPOCHS, verbose=1)

## 9. Análisis de métricas de rendimiento
-----

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

prec = history.history['precision']
val_prec = history.history['val_precision']

recall_1 = history.history['recall']
val_recall = history.history['val_recall']

f1 = history.history['f1_metric']
val_f1 = history.history['val_f1_metric']

epochs = range(1, len(acc) + 1)

# Pérdida de datos de entrenamiento y datos de prueba
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

plt.figure()

# Accuracy de los datos de entrenamiento y datos de prueba
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.figure()

# Precisión de los datos de entrenamiento y datos de prueba
plt.plot(epochs, prec, 'b', label='Training precision')
plt.plot(epochs, val_prec, 'r', label='Validation precision')
plt.title('Training and Validation precision')
plt.legend()

plt.figure()

# Recall de los datos de entrenamiento y datos de prueba
plt.plot(epochs, recall_1, 'b', label='Training recall')
plt.plot(epochs, val_recall, 'r', label='Validation recall')
plt.title('Training and Validation recall')
plt.legend()

plt.figure()

# F1 de los datos de entrenamiento y datos de prueba
plt.plot(epochs, f1, 'b', label='Training f1')
plt.plot(epochs, val_f1, 'r', label='Validation f1')
plt.title('Training and Validation f1')
plt.legend()

plt.show()


## 10. Evaluación de modelo
-----

In [ ]:
print("[INFO] Calcular la precisión del modelo....")
scores = model.evaluate(x_test, y_test)
print(f"Exactitud de la prueba: {scores[1]*100}")

## 11. Guardar modelo
-----

In [ ]:
# Guardar modelo
print("[INFO] Guardar modelo...")
model.save('model.h5')
#pickle.dump(model,open('cnn_model_a.pkl', 'wb'))


## 12. Realizar predicción
-----

In [ ]:
model = tf.keras.models.load_model('model.h5', compile=False)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy' ,tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

imgpath='Fotos_old/nosana.jpeg'

imar = convert_image_to_array(imgpath)
np_image_list = np.array([imar], dtype=np.float16) / 225.0

predicted_classes = model.predict(np_image_list)

for i, img_tagged in enumerate(predicted_classes):
    print(label_binarizer.classes_[img_tagged.tolist().index(max(img_tagged))])